# SF Bay Area Bike Share - Proyecto PAP

### Nombre: Luis Alberto Webster Gil
### Fecha: 06/10/2018
### ITESO A.C.

El presente archivo tiene como finalidad explicar al usuario el uso de bicicletas en el área de San Francisco y las ciudades cercanas a esta. Recordemos que el objetivo de estas bicicletas es resolver problemas de transporte para distancias cortas dentro de la misma ciudad. 

La estructura de este reporte se compone de la siguiente manera:

    1. Importación y obtención de datos
    2. Exploración de datasets
        2.1 Quality Report
        2.2 Gráficas descriptivas de las variables
        2.3 Comentarios basados en las gráficas generadas
    3. Modelo de Machine Learning
    4. Conclusiones

## 1. Importación y obtención de datos

Los datos utilizados fueron obtenidos de Kaggle (https://www.kaggle.com/benhamner/sf-bay-area-bike-share#trip.csv y se componen de 4 diferentes datasets que contienen datos desde Agosto del 2013 hasta Agosto del 2015:

    * station
    * status
    * trip
    * weather

Cada uno de estos datasets cuenta con un número diferente de registros y variables, lo cual añade un nivel de dificultad a la exploración de datos si se deseaba hacer de manera conjunta, es decir, en un solo dataframe. Con la finalidad de tener un mejor orden de los datasets y las variables a explorar, se decidió tener los 4 archivos csv en un diccionario llamado "docs". A continuación se presentan las líneas de código generadas para la importación de los datasets, paqueterías principales, work directory y funciones generadas por el usuario.

In [1]:
import numpy as np
import pandas as pd
import datetime
import sklearn.metrics as skm
import scipy.spatial.distance as sc
import matplotlib.pyplot as plt
import seaborn as sns
import os
from copy import deepcopy

#%% Path donde se está trabajando
path="C:\\Users\\LuisW\\Desktop\\ITESO\\XI Semestre\\Pap"
os.chdir(path) #os.getcwd() para verificar la ubicación del archivo a importar
os.getcwd()

#%% Funciones a utilizar en la base de datos
plt.style.use('ggplot')

PARAM_TRIP_TIME = 86  # seconds


def read_csv(csv_name):
    if '.csv' not in csv_name:
        csv_name += '.csv'
    doc = pd.read_csv(csv_name)
    return doc


def remove_unchanged_status(df, column_of_interest):
    df_copy = deepcopy(df)

    # Getting changes
    df_copy['not_change'] = df_copy[column_of_interest].eq(
        df_copy[column_of_interest].shift())

    # Removing unchanged
    df_copy = df_copy.loc[df_copy['not_change'] == 0]
    return df_copy


def transform_date(date, was_string=True):
    if was_string:
        return datetime.datetime.strptime(date, "%Y-%m-%d")
    return datetime.datetime.strftime(date, "%Y-%m-%d")


#%% Importar archivos csv en un diccionario
if __name__ == '__main__':
    csv_names = ['station', 'status', 'trip', 'weather']
    docs = {csv_name: read_csv(csv_name=csv_name) for csv_name in csv_names}

Cabe mencionar que las bases de datos tenían un peso aproximado de 660MB cuando estan comprimidas y de casi 5GB cuando se descomprimen. La base de datos con mayor número de registros fue status (aproximadamente 72 millones), razón por la cual se decidió crear un fragmento de código, el cual nos ayudaría a eliminar registros no necesarios del dataframe y así ganar velocidad en la ejecución del código en general. A continuación se presenta el código utilizado: 

In [2]:
def remove_unchanged_status(df, column_of_interest):
    df_copy = deepcopy(df)

    # Getting changes
    df_copy['not_change'] = df_copy[column_of_interest].eq(
        df_copy[column_of_interest].shift())

    # Removing unchanged
    df_copy = df_copy.loc[df_copy['not_change'] == 0]
    return df_copy

El dataset Status se redujo considerablemente ya que ahora tiene una dimensión de poco más de 2 millones de registros, agilizando así la ejecución del código. A continuación se encuentra el fragmento de código que nos ayuda a eliminar los registros innecesarios, utilizando la función mencionada anteriormente.

In [3]:
#%% Remover registros que no presentan cambio en Dataset Status.csv

docs['status'] = remove_unchanged_status(df=docs.get('status'),
                                             column_of_interest='bikes_available')

new_length_status=len(docs['status'].bikes_available)
new_length_status


2100422

## 2. Exploración de datasets

In [23]:
# funciones para crear el quality report (meterlas en un .py)

#importar funciones desde un archivo .py dentro de la misma carpeta)



#lista de variables en la base de datos
def columns_name(dataset):
    return pd.DataFrame(list(dataset.columns.values))

#lista de tipos de datos
def data_types(dataset):
    return pd.DataFrame(dataset.dtypes,columns=['Data Type'])

#lista de datos nulos o faltantes
def missing_values(dataset):
    return pd.DataFrame(dataset.isnull().sum(),columns=['Missing_Values'])

#número de datos en cada columna
def present_data(dataset):
    return pd.DataFrame(dataset.count(),columns=['Present_Data'])

#valores únicos en la base de datos
def unique_values(dataset):
    unique_values_dataset = pd.DataFrame(columns=['Unique_Values'])
    
    for col in list(dataset.columns.values):
        unique_values_dataset.loc[col]=[dataset[col].nunique()]
        
    return unique_values_dataset



#valores mínimos de las columnas
def minimum_values(dataset):
    minimum_values_dataset = pd.DataFrame(columns=['Minimum_Values'])
    
    for col in list(dataset.columns.values):
        try:
            minimum_values_dataset.loc[col]=[dataset[col].min()]
        except:
            pass
    return minimum_values_dataset



#valores máximos de las columnas
def maximum_values(dataset):
    maximum_values_dataset = pd.DataFrame(columns=['Maximum_Values'])
    
    for col in list(dataset.columns.values):
        try:
            maximum_values_dataset.loc[col]=[dataset[col].max()]
        except:
            pass
    return maximum_values_dataset





#Revisar función para crear el data quality report
#función para crear el data quality report
def data_quality_report(dataset):
    results= data_types.join(missing_values).join(present_data).join(unique_values).join(minimum_values).join(maximum_values)
    return results